## Conexión base de datos

In [11]:
import MySQLdb
import pandas as pd
from datetime import datetime

db = MySQLdb.connect(host="192.168.0.12",    # tu host, usualmente localhost
                     user="operario",         # tu usuario
                     passwd="",  # tu password
                     db="dbp8100")        # el nombre de la base de datos

# Debes crear un objeto Cursor. Te permitirá
# ejecutar todos los queries que necesitas
cur = db.cursor()

# Usa todas las sentencias SQL que quieras
cur.execute("SELECT * FROM tareaseje")

25169

# Extracción de datos

In [12]:
fecha_actual = datetime.now()
fecha_actual = fecha_actual.strftime("%Y-%m-%d")

In [13]:
# Extraer datos y convertir a df

# Datos mysql
cur.execute('''
            SELECT                 
                tareaseje.Fecha AS Fecha,
                tareaseje.Hora AS Hora,
                formulas.Nombre AS Nombre,
                tareaseje.Set AS Programado,
                (SELECT SUM(dcaptura.Valor) FROM dbp8100.dcaptura WHERE dcaptura.IDT = tareaseje.NroID) AS Dosificado,
                tareaseje.Tiempo AS Tiempo
            FROM 
                dbp8100.tareaseje AS tareaseje
            JOIN 
                dbp8100.formulas AS formulas ON formulas.NroID = tareaseje.IDF
            WHERE
                Fecha >= %s
            GROUP BY
                tareaseje.NroID
            ''', (fecha_actual,))
resultados = cur.fetchall()
# Obtener los nombres de las columnas
columnas = [desc[0] for desc in cur.description]

# Crear un DataFrame de pandas
df = pd.DataFrame(resultados, columns=columnas)

# Cerrar el cursor y la conexión
cur.close()
db.close()

# Mostrar el DataFrame
print(df)

         Fecha            Hora                                         Nombre  \
0   2025-02-06 0 days 03:08:28            RECRIA 14 SIN PELETEAR (quebradora)   
1   2025-02-06 0 days 05:16:27                       CREC. MELAZA 12 % A.A...   
2   2025-02-06 0 days 05:36:07                              LIMPIEZA CIRCUITO   
3   2025-02-06 0 days 05:43:34                     DESARROLLO (LA MALVINA)...   
4   2025-02-06 0 days 06:53:05                    TERMINACIÓN (LA MALVINA)...   
5   2025-02-06 0 days 07:28:09                    TERMINACIÓN (LA MALVINA)...   
6   2025-02-06 0 days 08:01:50                DESTETE PRECOZ MELAZA  10% A.A.   
7   2025-02-06 0 days 09:51:17                      INICIADOR MELAZA 14% A.A.   
8   2025-02-06 0 days 10:36:49                      INICIADOR MELAZA 14% A.A.   
9   2025-02-06 0 days 11:16:02                      VACA LECHERA 16 expeller.   
10  2025-02-06 0 days 12:02:08  (LA MALVINA) DESARROLLO TERNEROS MAIZ CON SAL   
11  2025-02-06 0 days 12:09:

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   Fecha       16 non-null     object         
 1   Hora        16 non-null     timedelta64[ns]
 2   Nombre      16 non-null     object         
 3   Programado  16 non-null     float64        
 4   Dosificado  16 non-null     float64        
 5   Tiempo      16 non-null     timedelta64[ns]
dtypes: float64(2), object(2), timedelta64[ns](2)
memory usage: 900.0+ bytes


# Convertir datos y graficar

In [15]:
#Corregir los tiempos 0
# Identificar filas con tiempo 0
mask = df['Tiempo'] == pd.Timedelta(0)

# Calcular la diferencia de hora con la siguiente fila
df.loc[mask, 'Tiempo'] = df['Hora'].shift(-1) - df.loc[mask, 'Hora']
df

,Fecha,Hora,Nombre,Programado,Dosificado,Tiempo
0,2025-02-06,0 days 03:08:28,RECRIA 14 SIN PELETEAR (quebradora),14000.0,14014.20,0 days 02:04:19
1,2025-02-06,0 days 05:16:27,CREC. MELAZA 12 % A.A...,2000.0,1993.35,0 days 00:15:37
2,2025-02-06,0 days 05:36:07,LIMPIEZA CIRCUITO,270.0,266.10,0 days 00:04:44
3,2025-02-06,0 days 05:43:34,DESARROLLO (LA MALVINA)...,12000.0,12005.10,0 days 01:09:02
4,2025-02-06,0 days 06:53:05,TERMINACIÓN (LA MALVINA)...,6000.0,1048.60,0 days 00:35:04
5,2025-02-06,0 days 07:28:09,TERMINACIÓN (LA MALVINA)...,4000.0,4008.80,0 days 00:26:24
6,2025-02-06,0 days 08:01:50,DESTETE PRECOZ MELAZA 10% A.A.,9990.0,10097.10,0 days 01:48:03
7,2025-02-06,0 days 09:51:17,INICIADOR MELAZA 14% A.A.,8000.0,4061.70,0 days 00:45:32
8,2025-02-06,0 days 10:36:49,INICIADOR MELAZA 14% A.A.,4000.0,4075.60,0 days 00:38:23
9,2025-02-06,0 days 11:16:02,VACA LECHERA 16 expeller.,4000.0,4050.95,0 days 00:43:45


In [16]:
df['Rendimiento'] = (df['Dosificado'] / (df['Tiempo'].astype('int64')/3.6e12))/1000
df

,Fecha,Hora,Nombre,Programado,Dosificado,Tiempo,Rendimiento
0,2025-02-06,0 days 03:08:28,RECRIA 14 SIN PELETEAR (quebradora),14000.0,14014.20,0 days 02:04:19,6.763791
1,2025-02-06,0 days 05:16:27,CREC. MELAZA 12 % A.A...,2000.0,1993.35,0 days 00:15:37,7.658549
2,2025-02-06,0 days 05:36:07,LIMPIEZA CIRCUITO,270.0,266.10,0 days 00:04:44,3.373099
3,2025-02-06,0 days 05:43:34,DESARROLLO (LA MALVINA)...,12000.0,12005.10,0 days 01:09:02,10.434177
4,2025-02-06,0 days 06:53:05,TERMINACIÓN (LA MALVINA)...,6000.0,1048.60,0 days 00:35:04,1.794183
5,2025-02-06,0 days 07:28:09,TERMINACIÓN (LA MALVINA)...,4000.0,4008.80,0 days 00:26:24,9.110909
6,2025-02-06,0 days 08:01:50,DESTETE PRECOZ MELAZA 10% A.A.,9990.0,10097.10,0 days 01:48:03,5.606904
7,2025-02-06,0 days 09:51:17,INICIADOR MELAZA 14% A.A.,8000.0,4061.70,0 days 00:45:32,5.352167
8,2025-02-06,0 days 10:36:49,INICIADOR MELAZA 14% A.A.,4000.0,4075.60,0 days 00:38:23,6.370890
9,2025-02-06,0 days 11:16:02,VACA LECHERA 16 expeller.,4000.0,4050.95,0 days 00:43:45,5.555589


## Gráficos

In [17]:
import plotly.express as px


In [25]:
nombre = df['Hora'].astype('str') + df['Nombre']

# Crear gráfico de barras
fig = px.bar(df, x=nombre, y='Dosificado', title='Producción por Producto')

# Mostrar el gráfico
fig.show()

In [29]:
# Crear gráfico de barras
fig = px.line(df, x='Hora', y='Rendimiento', title='Rendimiento por Producto')

# Mostrar el gráfico
fig.show()

In [23]:
import plotly.graph_objects as go

nombre = df['Hora'].astype('str') + df['Nombre']
# Crear gráfico de barras
fig = go.Figure(data=[go.Bar(x=nombre, y=df['Dosificado'])])

# Personalizar el gráfico
fig.update_layout(title='Produccion por Producto',
                  xaxis_title='Producto',
                  yaxis_title='Cantidad')

# Mostrar el gráfico
fig.show()
